# spark sharing


In [1]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import lit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer
from pyspark.sql.functions import regexp_replace

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer

import numpy as np
import pandas as pd

In [ ]:
1.basis
    1.1 SparkSession--DataFrame&SQL的入口
    1.2 DataFrame操作
    1.3 SQL的操作
    1.4 Estimator, Transformer, and Param
2.LR

3.GBDT

## 1、basis

### 1.1 SparkSession是DataFrame和SQL功能的主入口

In [2]:
# spark = SparkSession.builder.master("spark://10.0.120.106:7077").appName("spark_app").config("", "").getOrCreate()

spark = SparkSession.builder.master("local").appName("spark_app").config("", "").getOrCreate()

In [3]:
spark

### 1.2 读取数据文件

In [4]:
# 虽然是txt格式，但是按照csv来读取可以设置seperator
read_data = spark.read.csv('C:\spark\data\my_data\iris_data.txt', sep=',')
read_data.head(5)

[Row(_c0='5.1', _c1='3.5', _c2='1.4', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.9', _c1='3.0', _c2='1.4', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.7', _c1='3.2', _c2='1.3', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='4.6', _c1='3.1', _c2='1.5', _c3='0.2', _c4='Iris-setosa'),
 Row(_c0='5.0', _c1='3.6', _c2='1.4', _c3='0.2', _c4='Iris-setosa')]

### 1.3 拆分数据集

In [5]:
train_data, test_data = read_data.randomSplit([0.7, 0.3], 1)

In [6]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

### 1.4 操作dataframe

In [7]:
train_data.show(50)

+---+---+---+---+---------------+
|_c0|_c1|_c2|_c3|            _c4|
+---+---+---+---+---------------+
|4.3|3.0|1.1|0.1|    Iris-setosa|
|4.4|2.9|1.4|0.2|    Iris-setosa|
|4.4|3.0|1.3|0.2|    Iris-setosa|
|4.4|3.2|1.3|0.2|    Iris-setosa|
|4.6|3.1|1.5|0.2|    Iris-setosa|
|4.6|3.2|1.4|0.2|    Iris-setosa|
|4.6|3.4|1.4|0.3|    Iris-setosa|
|4.6|3.6|1.0|0.2|    Iris-setosa|
|4.7|3.2|1.3|0.2|    Iris-setosa|
|4.8|3.0|1.4|0.1|    Iris-setosa|
|4.8|3.0|1.4|0.3|    Iris-setosa|
|4.8|3.1|1.6|0.2|    Iris-setosa|
|4.8|3.4|1.6|0.2|    Iris-setosa|
|4.8|3.4|1.9|0.2|    Iris-setosa|
|4.9|2.5|4.5|1.7| Iris-virginica|
|4.9|3.1|1.5|0.1|    Iris-setosa|
|4.9|3.1|1.5|0.1|    Iris-setosa|
|5.0|2.0|3.5|1.0|Iris-versicolor|
|5.0|2.3|3.3|1.0|Iris-versicolor|
|5.0|3.2|1.2|0.2|    Iris-setosa|
|5.0|3.3|1.4|0.2|    Iris-setosa|
|5.0|3.4|1.5|0.2|    Iris-setosa|
|5.0|3.5|1.3|0.3|    Iris-setosa|
|5.0|3.5|1.6|0.6|    Iris-setosa|
|5.0|3.6|1.4|0.2|    Iris-setosa|
|5.1|3.3|1.7|0.5|    Iris-setosa|
|5.1|3.4|1.5|0

In [8]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [9]:
train_data.select('_c0')

DataFrame[_c0: string]

In [10]:
train_data.select('_c0').show(10)

+---+
|_c0|
+---+
|4.3|
|4.4|
|4.4|
|4.4|
|4.6|
|4.6|
|4.6|
|4.6|
|4.7|
|4.8|
+---+
only showing top 10 rows



In [29]:
train_data.select('_c0').collect()

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.6'),
 Row(_c0='4.7'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.8'),
 Row(_c0='4.9'),
 Row(_c0='4.9'),
 Row(_c0='4.9'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.0'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.1'),
 Row(_c0='5.2'),
 Row(_c0='5.2'),
 Row(_c0='5.3'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.4'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.5'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.6'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.7'),
 Row(_c0='5.8'),
 Row(_c0='5.8'),
 Row(_c0='5.8'),
 Row(_c0='5.8'

In [30]:
train_data.select('_c0').head(5)

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6')]

In [31]:
type(train_data.select('_c0').head(5))

list

In [32]:
train_data.select('_c0').take(5)

[Row(_c0='4.3'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.4'),
 Row(_c0='4.6')]

In [16]:
row = Row(value='5.1,3.5,1.4,0.2,Iris-setosa')

In [17]:
row

Row(value='5.1,3.5,1.4,0.2,Iris-setosa')

## 1、LR

## 2、GBDT

In [20]:
train_data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows



In [21]:
train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]